

*   Họ Tên: Huỳnh Thiện Tùng
*   MSSV: 19522492
*   Bài tập 12: bao gồm bài 12.3




# Import thư viện

In [65]:
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [66]:
!gdown --id 1NU0Yrx4yJyh3ny7InsejpuZBg7wf-Z67

Downloading...
From: https://drive.google.com/uc?id=1NU0Yrx4yJyh3ny7InsejpuZBg7wf-Z67
To: /content/loan_data.csv
100% 38.0k/38.0k [00:00<00:00, 34.9MB/s]


# Đọc và khám phá dữ liệu

In [89]:
df = pd.read_csv('loan_data.csv').dropna()
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [90]:
df.drop(df[df['Dependents'] == '3+'].index, inplace = True)
dataX = df.iloc[:, [3, 4, 5, 6, 7, 8, 9, 10, 11]]
dataY = df.iloc[:, 12].values
dataX = pd.get_dummies(data=dataX, columns=['Education', 'Self_Employed', 'Property_Area'])


In [91]:
print(dataX)
dataX = dataX.values

    Dependents  ApplicantIncome  ...  Property_Area_Semiurban  Property_Area_Urban
1            1             4583  ...                        0                    0
2            0             3000  ...                        0                    1
3            0             2583  ...                        0                    1
4            0             6000  ...                        0                    1
5            2             5417  ...                        0                    1
..         ...              ...  ...                      ...                  ...
608          0             3232  ...                        0                    0
609          0             2900  ...                        0                    0
611          1             8072  ...                        0                    1
612          2             7583  ...                        0                    1
613          0             4583  ...                        1                    0

[43

In [92]:
y = np.zeros_like(dataY, dtype=np.int)
# dataY = [1 if dataY == 'Y' else 0]
for idx, v in enumerate(dataY):
    if v == 'Y':
        y[idx] = 1
dataY = y
dataY

array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,

In [93]:
print(dataX.shape)
print(dataY.shape)

(439, 13)
(439,)


# Train/test spliting

In [94]:
x_train, x_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.2, shuffle=True)

In [95]:
print(x_train.shape)
print(x_test.shape)

(351, 13)
(88, 13)


# Data Normalization

In [96]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

# BÀI TẬP 1: GBM Model

In [97]:
model = GradientBoostingClassifier()
cv = KFold(n_splits=5, shuffle=True)
n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, error_score='raise')
print('Accuracy: %.3f' % (mean(n_scores)))

Accuracy: 0.803


# BÀI TẬP 2: Xgboost

In [98]:
model = XGBClassifier()
cv = KFold(n_splits=5, shuffle=True)
n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv)
print('Accuracy: %.3f' % (mean(n_scores)))

Accuracy: 0.792
